In [1]:
import dotenv
dotenv.load_dotenv()
import openai
import os

In [2]:
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

In [3]:
import pandas as pd
import evaluate
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
import os
import getpass
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.llms import HuggingFaceHub

/usr/local/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = 'What classes does the boating center offer to community members?'

In [5]:
context = """
UC Santa Cruz Community Boating Center is a community-based boating program that connects students and community members to the Monterey Bay through recreational activities including sailing, rowing and kayaking.
UC Santa Cruz Community Boating Center staff and veteran participants provide a familiarity with the equipment, skills, and water safety that comes from a lifetime of enjoying these sports.
The Community Boating Center also connects people who are interested in these sports to one another, whether you are a beginner seeking to learn something new or a experienced sailor, rower, and/or kayaker looking to improve your skills and get time on the water. Whether you've never been on a boat before, or want to build on old skills, we have a class for you! From 2-week junior summer camps to quarterly physical education classes for UC Santa Cruz Students, you'll be cruising around Monterey Bay with ease.

Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.
Our boats:
Sailing vessels for weekend use currently include RS Quests, Lasers, RS Visions, a SC27, a Santana 22, a J 24, an Olson 30, and an O'Day 34.
The rowing fleet exists of eight Edon training shells, two MAAS AEROS 20s, two MAAS 24s, Bay 21, Pegasus, WinTech 30' double, and a custom-built wooden double. 
Want to take a boat out? Make sure to pay your membership dues and boat fees online ahead of time. The boating center will no longer be accepting cash payments on-site.
Sailing Hours:
Friday 4:00-7:00 PM (during daylight savings)
Saturday 12:00-5:00 PM
Sunday 12:00-5:00 PM
Members of the Community Boating Club have access to sailboats and rowing shells during club hours. Members must be approved for boat use by the boating directors before checking out a rowing shall or sailboat. Check our calendar for the next available checkout day. If you do not have prior sailing or rowing experience, we offer classes to build your confidence on the water! For class information, click here.
Sailors and rowers are evaluated based on experience and allowed boat use accordingly.

Boating Club Hours
The Community Boating Center is open 2 days a week.
Saturday: 12:00 PM - 5:00 PM
Sunday: 12:00 PM - 5:00 PM

How can I join the Boating Club?
"""

In [24]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": f"Given this query and context, return the query in logical english (LE). Context: {context}, Query: {query}. Use as little words as possible."}
    ]
)
le = response.choices[0].message.content
print(le)

The Boating Center offers recreational classes for sailing, rowing and kayaking, ranging from beginner to experienced levels. It also provides junior summer camps and physical education classes for UC Santa Cruz Students.


In [28]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": f"Convert this logical english sentence to a prolog database to answer a query. LE: {le}, Query: {query}. Only return prolog without an intro."}
    ]
)
prolog = response.choices[0].message.content
print(prolog)

classes_offered_by_boating_center(recreational_sailing).
classes_offered_by_boating_center(recreational_rowing).
classes_offered_by_boating_center(recreational_kayaking).
classes_offered_by_boating_center(beginner).
classes_offered_by_boating_center(experienced).
classes_offered_by_boating_center(junior_summer_camps).
classes_offered_by_boating_center(physical_education_for_ucsc_students). 

% The query would be:
% ?- classes_offered_by_boating_center(Class).


In [8]:
def strip_markdown_code_block(text):
    if text.startswith("```"):
        lines = text.strip().splitlines()
        # Remove first and last lines (e.g. ```prolog and ```)
        return "\n".join(lines[1:-1])
    return text

In [9]:
clean_prolog = prolog
print(clean_prolog)

classes(X).
offers(uc_santa_cruz_community_boating_center, X) :-
  classes(X).
for(X, community_members) :-
  classes(X).
classes(sailing).
classes(rowing).
classes(kayaking).


In [10]:
with open("rules.pl", "w") as f:
    f.write(clean_prolog)

In [11]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": f"Here I have a prolog rule and query, return the prolog query to run to get all correct responses. Query: {query}, Rulebase: {clean_prolog} Only output the prolog query. Format the query as such: Query, wite(variable), nl"}
    ]
)
prolog_query = response.choices[0].message.content
print(prolog_query)

?- for(Class, community_members), write(Class), nl.


In [12]:
# print(prolog_query[3:-1])

In [13]:
import subprocess

# File containing Prolog code
prolog_file = "rules.pl"
pl_query = prolog_query #[3:-1]

# This assumes your .pl file defines a predicate called `query/0`
result = subprocess.run(
    ["swipl", "-s", prolog_file, "-g", pl_query, "-t", "halt"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("STDOUT:")
print(result.stdout)
print("STDERR:")
print(result.stderr)


STDOUT:

STDERR:
ERROR: -g ?- for(Class, community_members), write(Class), nl.: catch/3: Unknown procedure: (?-)/1
ERROR:   ?- is the Prolog prompt
ERROR:   See FAQ at https://www.swi-prolog.org/FAQ/ToplevelMode.html



In [14]:
from pyswip import Prolog

prolog = Prolog()
prolog.consult("rules.pl")

# Run a query
results = list(prolog.query(c_query))

# Print the results
for result in results:
    print(result)


TypeError: search() missing 1 required positional argument: 'string'